In [133]:
# Import the dependencies.

from flask import Flask, json, jsonify
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect

In [134]:
#################################################
# Database Setup
#################################################

engine = create_engine("sqlite:///../Resources/Starter_Code/Resources/hawaii.sqlite")

In [135]:
# reflect an existing database into a new model

Base = automap_base()

In [136]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [137]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station


In [138]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [139]:
#################################################
# Flask Setup
#################################################

app = Flask(__name__)

In [140]:
#################################################
# Flask Routes
#################################################

#@app.route("/")



In [141]:
#1 
#Start at the homepage.
#List all the available routes.

@app.route("/")

def welcome_page():
    
    print("Homepage.")
    return (
       f"Welcome to the Climate API!:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start<br/>"
        f"/api/v1.0/start/end"
    
    )

In [142]:
#2
#/api/v1.0/precipitation

#Convert the query results from your precipitation analysis (i.e. retrieve only the last 12 months of data) to a dictionary using date as the key and prcp as the value.
#Return the JSON representation of your dictionary.

@app.route('/api/v1.0/precipitation/')
def precipitation():
    
    session = Session(engine)
    
    print("In Precipitation section.")
    
    year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)

    prcp_results = session.query(measurement.date, measurement.prcp).\
        filter(measurement.date >= year_ago).\
        order_by(measurement.date).all()
    
    session.close()

    prcp_dict = {date: prcp for date, prcp in prcp_results}
    
    
   
    return jsonify(prcp_dict) 

In [144]:
#3
#/api/v1.0/stations
#Return a JSON list of stations from the dataset.

@app.route("/api/v1.0/stations")

def stations():
    
    session = Session(engine)
    
    station_results = session.query(station.station).all()
    
    session.close()
    
    station_list = []
    
    for station in station_results:
        station_dict = {}
        station_dict["Station"] = station.station
        station_dict["Name"] = station.name
        
        station_list.append(station_dict)
        
    return jsonify(station.list)
        

In [145]:
#4
#/api/v1.0/tobs

#Query the dates and temperature observations of the most-active station for the previous year of data.
#Return a JSON list of temperature observations for the previous year.

@app.route("/api/v1.0/tobs")

def tobs():
    
    session = Session(engine)

        
    year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
        
    active_results = session.query(measurement.date, measurement.tobs).\
        filter(measurement.station == 'USC00519281').\
        filter(measurement.date >= year_ago).all()
    
    session.close()

    
    tobs_list = []
    
    for date, tobs in active_results:
        tobs_dict = {}
        tobs_dict['date'] = date
        tobs_dict['tobs'] = tobs
        tobs_list.append(tobs_dict)
        
    return jsonify(tobs_list)
        

In [147]:
#5
#/api/v1.0/<start> and /api/v1.0/<start>/<end>

#Return a JSON list of the minimum temperature, the average temperature, and the maximum temperature for a specified start or start-end range.
#For a specified start, calculate TMIN, TAVG, and TMAX for all the dates greater than or equal to the start date.
#For a specified start date and end date, calculate TMIN, TAVG, and TMAX for the dates from the start date to the end date, inclusive.

@app.route("/api/v1.0/<start>")

def temperature(start):
    
    session = Session(engine)

    select_start = [func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)]
    
    temp_results = session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
        filter(measurement.date >= start).all()
    
    session.close()
    
    temp_observed = {}
    temp_observed["Min_Temp"] = temp_results[0][0]
    temp_observed["avg_Temp"] = temp_results[0][1]
    temp_observed["max_Temp"] = temp_results[0][2]
    
    return jsonify(temp_observed)
    
@app.route("/api/v1.0/start/end")

def temperature_start_end(start, end):
    
     session = Session(engine)
        
        result = session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
                               filter(measurement.date >= start).filter(measurement.date <= end).all()
        
        session.close()
        
        temp_observed["Min_temp"] = result[0][0]
        temp_observed["avg_temp"] = result[0][1]
        temp_observed["max_temp"] = result[0][2]
        
        return jsonify(temp_observed)
    
if __name__ == '__main__':
    
    app.run(debug=True)
    
    
    
    
